In [1]:
#load libraries and setup file
import sys,os
root_path = os.path.abspath(os.path.join('..'))
print(root_path)
if root_path not in sys.path:
    sys.path.append(root_path)
%run -i "../lib/utils/ipynb_setup.py"
%load_ext autoreload
%autoreload 2
%matplotlib inline

/home/mptouzel/Dropbox/scripts/Projects/topicmod/code


/home/mptouzel/Dropbox/scripts/Projects/topicmod/code/lib/utils/ipynb_setup.py:15: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  pl.rc('text.latex', preamble=[r'\usepackage{amsmath}'])
/home/mptouzel/Dropbox/scripts/Projects/topicmod/code/lib/utils/ipynb_setup.py:27: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#plotting
import matplotlib.pyplot as pl
import seaborn as sns
sns.set_style("ticks", {'axes.grid' : True})
# pl.rc("figure", facecolor="white",figsize = (8,8))
pl.rc("figure", facecolor="gray",figsize = (8,8))
pl.rc('text', usetex=True)
pl.rc('text.latex', preamble=[r'\usepackage{amsmath}'])
pl.rc('lines',markeredgewidth = 2)
pl.rc('font',size = 12)

/tmp/ipykernel_90079/2519538835.py:8: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  pl.rc('text.latex', preamble=[r'\usepackage{amsmath}'])


## load carbon tax data

In [3]:
df=pd.read_stata('../../data/carbon_tax/CCOP_wave1.dta')
df.head(1)

,responseid,status,intcode,interview_start,interview_end,temptime,lang,prov,d1,d3,...,d7,d8,d10,d11,d12,d13,d14,immig,kids,Weight
0,27,complete,C1,2019-02-22 11:27:04,2019-02-22 11:54:52,1665,EN,ON,Male,63,...,0,2,"Within a suburb, adjacent to a large city",8,College / CEGEP graduate,This doesn't apply to me,"$40,000-$59,999","No, Born in Canada",No,2.389879


Select data to analyze

In [4]:
carbon_tax_question='po3a'
participant_sex='d1'
french_response_ids=df[df.lang=='FR'].index

question_id_name=carbon_tax_question
meta_data_names=[]#[participant_sex]
exclusions=french_response_ids #english only

data=df.drop(exclusions).loc[:,[question_id_name]+meta_data_names].copy()

Run custom corpus cleaning

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mptouzel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from lib.lib import clean_corpus
wordfixfile='../../data/carbon_tax/CCOP_wave1_nonwords.csv'
customclean_data=clean_corpus(data,carbon_tax_question,wordfixfile=wordfixfile)

/home/mptouzel/Dropbox/scripts/Projects/topicmod/code/lib/lib.py:97: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dlist=list(set(list(d_series.apply(pd.Series).stack().apply(lambda x: x[0] if isinstance(x,list) else x).values)))


# STM

## Preprocessing

In R, so need to interface

In [7]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
#N.b. need to install all packages to get stm 1.3.6 working in R first

In [8]:
for row in rpy2.situation.iter_info():
    print(row)

rpy2 version:
3.4.5
Python version:
3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Looking for R's HOME:
    Environment variable R_HOME: /usr/lib/R
    Calling `R RHOME`: /usr/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server
R version:
    In the PATH: R version 4.1.3 (2022-03-10) -- "One Push-Up"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/usr/share/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'rt', 'dl', 'm', 'icuuc', 'icui18n']
  library_dirs:
  ['/usr/lib/R/lib']
  extra_compile_args:
  []
  extra_link_args:
  ['-Wl,--export-dynamic', '-fopenmp', '-Wl,-Bsymbolic-functions', '-Wl,-z,relro']


In [11]:
# R library imports
for __pkg__ in ['Matrix', 'stringr', 'splines', 'matrixStats', 
    'slam', 'lda', 'glmnet', 'magrittr','stm']:
    try:
        robjects.r("library(" + __pkg__ + ")")
    except:
        raise Exception("Missing R library " + __pkg__ + 
            ". Please ensure the R requirements are satisfied.")

R[write to console]: Loaded glmnet 4.1-3



In [9]:
robjects.r("library('stm')")

R[write to console]: stm v1.3.6 successfully loaded. See ?stm for help. 
 Papers, resources, and other materials at structuraltopicmodel.com



'stm','tools','stats',...,'datasets','methods','base'


load data into R

In [10]:
from rpy2.robjects.conversion import localconverter

In [11]:
with localconverter(robjects.default_converter + pandas2ri.converter):
    robjects.globalenv['doc_df'] = robjects.conversion.py2rpy(data)
robjects.globalenv['doc_df']

,po3a
0,
1,
2,
3,
4,
...,...
3308,
3309,
3310,Great
3311,hopefully it does help to reduce climate change


Run STM's corpus cleaning

In [12]:
min_occur=1
robjects.r("processed_corpus_temp = textProcessor(doc_df$" + question_id_name +", metadata=doc_df, lowercase=TRUE)");
robjects.r(
    'processed_corpus = prepDocuments(processed_corpus_temp$documents,'+\
                                 'processed_corpus_temp$vocab,'+\
                                 'processed_corpus_temp$meta,'+\
                                 'lower.thresh='+str(min_occur)+') \n'+\
    'rm(processed_corpus_temp); invisible(gc())'
);

Building corpus... 
Converting to Lower Case... 
Removing punctuation... 
Removing stopwords... 
Removing numbers... 
Stemming... 
Creating Output... 
Removing 685 of 1326 terms (685 of 8500 tokens) due to frequency 
Removing 12 Documents with No Words 
Your corpus now has 1013 documents, 641 terms and 7815 tokens.

In [17]:
preprocessing_output=dict(robjects.r.processed_corpus.items())
print(list(preprocessing_output.keys()))
stmclean_dictionary=pd.Series(list(preprocessing_output['vocab']),name='word')
stmclean_corpus=pd.DataFrame.from_records([doc for doc in preprocessing_output['documents'].items()],index=[0]).drop(0,axis=1)
stmclean_corpus.index.name= None
stmclean_corpus.rename(columns={1: question_id_name+'clean_stems_wordvec'},inplace=True)
stmclean_corpus[question_id_name+'clean_stems_wordvec']=stmclean_corpus[question_id_name+'clean_stems_wordvec'].apply(lambda x:x[0])
extra_columns=dict(preprocessing_output['meta'].items())
for column_name,column in extra_columns.items():
    stmclean_corpus[column_name]=pd.Series(list(column),index=stmclean_corpus.index)
stmclean_data=(stmclean_corpus,stmclean_dictionary)

['documents', 'vocab', 'meta', 'words.removed', 'docs.removed', 'tokens.removed', 'wordcounts']


In [18]:
stmclean_corpus

,po3aclean_stems_wordvec,po3a
7,"[48, 95, 117, 131, 134, 187, 248, 300, 357, 36...",It will get people to think about their consum...
8,"[25, 29, 183, 195, 208, 227, 235, 238, 293, 34...",Phasing out the use of fossil fuels is essenti...
10,"[342, 424, 622, 630]",People will respect it more and be less wasteful
12,"[7, 60, 86, 99, 130, 186, 190, 213, 235, 238, ...",I believe putting price or tax on fossil fuels...
15,"[13, 56, 310, 408, 410, 424, 455]","As price increases, people are more open to op..."
...,...,...
2734,"[86, 99, 157, 332, 559, 623]",I don't know of any other way to stop climate ...
2735,"[15, 52, 60, 106, 116, 157, 240, 252, 296, 414...",I do believe it is a good idea overall. The re...
2741,"[45, 86, 89, 99, 197, 389, 424, 431, 451, 588,...",As thinking people we MUST do things to avoid ...
2753,[259],Great


## compare the two corpii

In [19]:
for tmpdata in [customclean_data,stmclean_data]:
    corpus,worddict=tmpdata
    print('n_resp={}, n_dict={}'.format(len(corpus),len(worddict)))
    print(list(corpus.columns.values))
    corpus.po3aclean_stems_wordvec.head()

n_resp=1047, n_dict=1192
['po3a', 'po3aclean', 'po3aclean_stems', 'po3aclean_stems_wordvec']
n_resp=1013, n_dict=641
['po3aclean_stems_wordvec', 'po3a']


## Define model & inference parameters

In [13]:
K=5
init_type="Spectral"
seed=None
max_em_its=500
emtol=1e-5
verbose=False#True
reportevery=5
LDAbeta=True
interactions=True
ngroups=1
gamma_prior="Pooled"
sigma_prior=0
kappa_prior="L1"
# model_parameters={
#     'K': 10,  # number of topics
#     'LDAbeta': LDAbeta,
#     'gamma_prior':,
#     'sigma_prior':,
#     'kappa_prior':
# }
# inference_options = {
#     'init_type': init_type,
#     'seed':,
#     'max_em_its':,
#     'emtol':,
#     'verbose':,
#     'ngroups':1,
# }

## Run inference (no prevalence or content)

Initialize

In [21]:
for __f__ in ['checkBeta.R', 'checkFactors.R', 'cloud.R', 'dmr.R', 
    'estimateEffect.R', 'exclusivity.R', 'findThoughts.R', 'findTopic.R', 
    'heldout.R', 'jeffreyskappa.R', 'labelTopics.R', 'manyTopics.R', 
    'multiSTM.R', 'parseFormulas.R', 'permute.R', 'plot.estimateEffect.R', 
    'plot.searchK.R', 'plot.STM.R', 'plotModels.R', 'plotQuote.R', 
    'plotRemoved.R', 'plottingutilfns.R', 'plotTopicLoess.R', 'prepDocuments.R',
    'produce_cmatrix.R', 'RcppExports.R', 'readCorpus.R', 'residuals.R', 's.R',
    'sageLabels.R', 'searchK.R', 'selectModel.R', 'semanticCoherence.R', 
    'simBetas.R', 'spectral.R', 'stm.control.R', 'stm.R', 'STMconvergence.R', 
    'STMestep.R', 'STMfunctions.R', 'STMinit.R', 'STMlncpp.R', 'STMmnreg.R', 
    'STMmu.R', 'STMoptbeta.R', 'STMreport.R', 'STMsigma.R', 'summary.STM.R', 
    'tau.R', 'textProcessor.R', 'thetaPosterior.R', 'toLDAvis.R', 'topicCorr.R', 
    'topicLasso.R', 'topicQuality.R', 'writeLdac.R']:
    robjects.r("source('/home/mptouzel/R/x86_64-pc-linux-gnu-library/4.1/stm/R/"+__f__+"')")

In [14]:
yield_seed = lambda x: robjects.NULL if x==None else x
robjects.globalenv["fit"] = robjects.r.stm(
    robjects.r("processed_corpus$documents"), 
    robjects.r("processed_corpus$vocab"), 
    K=K, data=robjects.r("processed_corpus$meta"),
    init_type=init_type, seed=yield_seed(seed),max_em_its=max_em_its)
#, emtol=emtol,
#     verbose=verbose, reportevery=reportevery,
#     LDAbeta=LDAbeta, interactions=interactions,
#     ngroups=ngroups, gamma_prior=gamma_prior,
#     sigma_prior=sigma_prior, kappa_prior=kappa_prior)

Beginning Spectral Initialization 
	 Calculating the gram matrix...
	 Finding anchor words...
 	.....
	 Recovering initialization...
 	......
Initialization complete.
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 1 (approx. per word bound = -5.855) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 2 (approx. per word bound = -5.672, relative change = 3.133e-02) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 3 (approx. per word bound = -5.592, relative change = 1.400e-02) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-

.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 30 (approx. per word bound = -5.487, relative change = 1.129e-04) 
Topic 1: dont, increas, good, also, incent 
 Topic 2: tax, help, energi, money, pay 
 Topic 3: will, peopl, fuel, carbon, emiss 
 Topic 4: use, reduc, need, make, think 
 Topic 5: fossil, chang, climat, can, put 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 31 (approx. per word bound = -5.487, relative change = 1.124e-04) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 32 (approx. per word bound = -5.486, relative change = 1.112e-04) 
...................................................................

.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 59 (approx. per word bound = -5.469, relative change = 9.457e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 60 (approx. per word bound = -5.469, relative change = 9.122e-05) 
Topic 1: dont, increas, good, futur, also 
 Topic 2: tax, help, energi, money, pay 
 Topic 3: will, peopl, fuel, carbon, emiss 
 Topic 4: use, reduc, need, make, think 
 Topic 5: fossil, chang, climat, can, put 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 61 (approx. per word bound = -5.468, relative change = 8.851e-05) 
....................................................................

.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 88 (approx. per word bound = -5.455, relative change = 7.585e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 89 (approx. per word bound = -5.455, relative change = 7.713e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 90 (approx. per word bound = -5.454, relative change = 7.839e-05) 
Topic 1: dont, increas, good, believ, futur 
 Topic 2: tax, help, energi, money, pay 
 Topic 3: will, peopl, fuel, carbon, emiss 
 Topic 4: use, reduc, need, make, think 
 Topic 5: fossil, chang, climat, can, put 
..................................................................

.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 117 (approx. per word bound = -5.439, relative change = 1.156e-04) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 118 (approx. per word bound = -5.439, relative change = 1.097e-04) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 119 (approx. per word bound = -5.438, relative change = 1.055e-04) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 120 (approx. per word bound = -5.437, relative change = 1.028e-04) 
Topic 1: dont, increas, good, thing,

 Topic 5: chang, climat, need, can, put 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 146 (approx. per word bound = -5.428, relative change = 3.684e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 147 (approx. per word bound = -5.428, relative change = 3.930e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 148 (approx. per word bound = -5.428, relative change = 4.039e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 149 (approx. per word bound = -5.427, relative change = 4.013e-

Completed M-Step. 
Completing Iteration 175 (approx. per word bound = -5.424, relative change = 1.988e-05) 
Topic 1: dont, increas, good, thing, also 
 Topic 2: tax, help, energi, money, pay 
 Topic 3: will, peopl, fuel, carbon, fossil 
 Topic 4: use, reduc, make, think, way 
 Topic 5: chang, climat, need, can, put 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 176 (approx. per word bound = -5.423, relative change = 2.008e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 177 (approx. per word bound = -5.423, relative change = 2.069e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 178 (approx. per word bound

.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 204 (approx. per word bound = -5.419, relative change = 3.045e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 205 (approx. per word bound = -5.418, relative change = 2.822e-05) 
Topic 1: dont, increas, good, thing, also 
 Topic 2: tax, help, energi, money, pay 
 Topic 3: will, peopl, fuel, carbon, fossil 
 Topic 4: use, reduc, make, think, way 
 Topic 5: chang, climat, need, can, effect 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 206 (approx. per word bound = -5.418, relative change = 2.558e-05) 
................................................................

.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 233 (approx. per word bound = -5.415, relative change = 2.892e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 234 (approx. per word bound = -5.415, relative change = 2.904e-05) 
.....................................................................................................
Completed E-Step (0 seconds). 
Completed M-Step. 
Completing Iteration 235 (approx. per word bound = -5.415, relative change = 2.831e-05) 
Topic 1: dont, like, increas, good, thing 
 Topic 2: tax, help, energi, money, pay 
 Topic 3: will, peopl, fuel, carbon, fossil 
 Topic 4: use, reduc, make, think, way 
 Topic 5: chang, climat, need, can, effect 
................................................................

In [15]:
print(list(robjects.r.fit.names))

['mu', 'sigma', 'beta', 'settings', 'vocab', 'convergence', 'theta', 'eta', 'invsigma', 'time', 'version']


In [18]:
dict(robjects.r('labelTopics(fit)').items())['frex']

'dont','energi','less',...,'fuel','current','warm'


## code junkyard

In [43]:
# Finish initialization
robjects.r('''

    # Extract objects
    documents <- fit$documents
    vocab <- fit$vocab
    settings <- fit$settings 
    model <- fit$model
    verbose <- settings$verbose

    # Initialize parameters
    ngroups <- settings$ngroups
    if(is.null(model)) {

        if(verbose) cat("Beginning Initialization.\n")
        model <- stm.init(documents, settings)

        # If using the Lee and Mimno method of setting K, update the settings
        if(settings$dim$K==0) settings$dim$K <- nrow(model$beta[[1]])

        # Unpack
        mu <- list(mu=model$mu)
        sigma <- model$sigma
        beta <- list(beta=model$beta)
        if(!is.null(model$kappa)) beta$kappa <- model$kappa
        lambda <- model$lambda
        convergence <- NULL 

        # Discard the old object
        # rm(model)
    } else {

        if(verbose) cat("Restarting Model...\n")

        # Extract from a standard STM object so we can simply continue
        mu <- model$mu
        beta <- list(beta=lapply(model$beta$logbeta, exp))
        if(!is.null(model$beta$kappa)) beta$kappa <- model$beta$kappa
        sigma <- model$sigma
        lambda <- model$eta
        convergence <- model$convergence

        # Manually declare the model not converged or it will stop after the first iteration
        convergence$stopits <- FALSE
        convergence$converged <- FALSE

        # Iterate by 1 as that would have happened otherwise
        convergence$its <- convergence$its + 1 
    }    

    #Pull out some book keeping elements
    ntokens <- sum(settings$dim$wcounts$x)
    betaindex <- settings$covariates$betaindex
    stopits <- FALSE
    if(ngroups!=1) {
        groups <- cut(1:length(documents), breaks=ngroups, labels=FALSE) 
    }
    suffstats <- vector(mode="list", length=ngroups)
''');

# Get the settings
fit = dict(zip(robjects.globalenv['fit'].names, 
         list(robjects.globalenv['fit'])))
settings = dict(zip( fit['settings'].names, 
         list(fit['settings'])))
modK, modA, modV, modN = [int(settings['dim'][i][0]) for i in range(4)]

# Some setup for EM, retrieving the R objects
stopits = False
documents = [np.array(x) for x in list(robjects.globalenv['documents'])]
beta_index = np.array(robjects.globalenv['betaindex'])
beta = [np.array(x) for x in robjects.globalenv['beta'][0]]
Lambda = np.array(robjects.globalenv['lambda'])
def rmu_2py(mu):
    out = {'mu': np.array(mu[0])}
    if len(mu) > 1:
        out['gamma'] = np.array(mu[1])
    else:
        out['gamma'] = None
    return out
mu = rmu_2py(robjects.globalenv['mu'])
sigma = np.array(robjects.globalenv['sigma'])
verbose = settings['verbose'][0]

# Clear the convergence object
try:
    del convergence
    i = 0
except:
    i = 0


R[write to console]: Error in rep(1:length(documents), times = VsubD) : 
  invalid 'times' argument



RRuntimeError: Error in rep(1:length(documents), times = VsubD) : 
  invalid 'times' argument


Run training loop

In [ ]:
# Initiate timer
start_time = time.clock()

# Run EM!
while stopits is not True:

    # Non-blocked updates
    if ngroups==1:

        # Run the E-step
        sigma_ss, beta_ss, bound_ss, Lambda = self.__estep_spark__(documents, beta_index, beta, 
                                                          Lambda, mu, sigma, verbose, False)
        if verbose:
            print("Completed E-Step, Iteration " +  str(i+1))

        # Run the M-step
        mu = self.__rmu_2py__(robjects.globalenv['opt.mu'](Lambda, 
                             robjects.r("settings$gamma$mode"),
                             robjects.r("settings$covariates$X"),
                             robjects.r("settings$gamma$enet")))
        sigma = np.array(robjects.r("opt.sigma")(sigma_ss, 
                                                 Lambda, 
                                                 mu['mu'], 
                                                 float(robjects.r("settings$sigma$prior"))))
        beta_new_temp = robjects.globalenv['opt.beta'](self.__pybeta_ss_2r__(beta_ss), 
                                                       robjects.r('beta$kappa'), 
                                                       robjects.r('settings'))
        beta = [np.array(x) for x in beta_new_temp[0]]

        if verbose:
            print("Completed M-Step, Iteration " +  str(i+1))

    # Check for convergence
    try:
        convergence = robjects.globalenv['convergence.check'](bound_ss, convergence, 
                                                              robjects.globalenv['settings'])
    except:
        convergence = robjects.globalenv['convergence.check'](bound_ss, 
                                                              robjects.globalenv['convergence'], 
                                                              robjects.globalenv['settings'])

    # Print updates if we have not converged
    stopits = bool(self.__rlist_2py__(convergence)['stopits'][0])
    if stopits is not True and verbose:
        robjects.r.report(convergence, robjects.r.ntokens, self.__pybeta_2r__(beta),
                 robjects.r.vocab, robjects.r('settings$topicreportevery'),
                 verbose);

    # Increase counter
    i += 1
    if verbose:
        print("")

# Some book-keeping
self.em_time = time.clock() - start_time
self.trained = True

# Save the results
self.beta = beta
self.logbeta = [np.log(x) for x in self.beta]
self.Lambda = Lambda
self.mu = mu
self.sigma = sigma
self.settings = settings
self.vocab = robjects.r.vocab
self.convergence = convergence
self.theta = np.array(robjects.r("theta_from_lambda")(Lambda))
self.eta = np.array(robjects.r("eta_from_lambda")(Lambda))
self.invsigma = np.linalg.inv(sigma)

In [ ]:
def train(self,data, document_col, K, prevalence=None, content=None, init_type="Spectral", 
    seed=None, max_em_its=500, emtol=1e-5, verbose=True, reportevery=5, 
    LDAbeta=True, interactions=True, ngroups=1, gamma_prior="Pooled", 
    sigma_prior=0, kappa_prior="L1"):
    '''
    Train a STM model

    Parameters:
      data - Pandas dataframe
        The data, in the form of a Pandas dataframe

      document_col - string
        Name of the data column containing the raw text 

      K - int 
        Number of topics

      prevalence - string | list of strings
        Name or list of names of columns containing prevalence covariates

      content - string | list of strings
        Name or list of names of columns containing content covariates 

      init_type - string
        The method of initialization. Must be either Latent Dirichlet Allocation 
        ("LDA"), "Random" or "Spectral".

      seed - int
        Random seed

      max_em_its - int 
        The maximum number of EM iterations. If convergence has not been met 
        at this point, a message will be printed.

      emtol - float
        Convergence tolerance. EM stops when the relative change in the approximate 
        bound drops below this level. Defaults to .00001.

      verbose - bool
        A logical flag indicating whether information should be printed to the 
        screen. During the E-step (iteration over documents) a dot will print 
        each time one percent of the documents are completed. At the end of each 
        iteration the approximate bound will also be printed.

      reportevery - int
        An integer determining the intervals at which labels are printed to 
        the screen during fitting. Defaults to every 5 iterations.

      LDAbeta - bool
        A logical that defaults to True when there are no content covariates. 
        When set to FALSE the model performs SAGE style topic updates (sparse 
        deviations from a baseline).

      interactions - bool
        A logical that defaults to TRUE. This automatically includes interactions 
        between content covariates and the latent topics. Setting it to FALSE 
        reduces to a model with no interactive effects.

      ngroups - int
        Number of groups for memoized inference.

      gamma_prior - string
        Sets the prior estimation method for the prevalence covariate model. 
        The default Pooled options uses Normal prior distributions with a 
        topic-level pooled variance which is given a broad gamma hyperprior. 
        The alternative L1 uses glmnet to estimate a grouped penalty between L1-L2.

      sigma_prior - int
        A scalar between 0 and 1 which defaults to 0. This sets the strength of 
        regularization towards a diagonalized covariance matrix. Setting the value 
        above 0 can be useful if topics are becoming too highly correlated.

      kappa_prior - int
        Sets the prior estimation for the content covariate coefficients. 
        The default option is the L1 prior. The second option is Jeffreys 
        which is markedly less computationally efficient but is included for 
        backwards compatability.
    '''

    # Blocked updates are not yet supported
    if ngroups > 1:
        raise Exception("Blocked inference not yet supported. ")

    # Push the data to R
    pandas2ri.activate()
    try:
        robjects.globalenv['doc_df'] = pandas2ri.py2ri(data)
    except:
        raise Exception("The data must be in the form of a Pandas dataframe")

    # Parse any date objects
    robjects.r('''
        if("date" %in% colnames(doc_df)){
            doc_df$date = as.Date(doc_df$date)
        }
    ''');

    # Prep the corpus
    print("Beginning document preprocessing.")
    robjects.r("processed_corpus_temp = textProcessor(doc_df$" + document_col + 
        ", metadata=doc_df, lowercase=TRUE)");
    robjects.r('''
        processed_corpus = prepDocuments(processed_corpus_temp$documents,
                                     processed_corpus_temp$vocab, 
                                     processed_corpus_temp$meta,
                                     lower.thresh=1)
        rm(processed_corpus_temp); invisible(gc())
    ''');
    print("")

    # Seed parsing
    yield_seed = lambda x: robjects.NULL if x==None else x

    # Initialize the run
    if prevalence != None and content != None:
        robjects.globalenv["fit"] = robjects.r.stm(
            robjects.r("processed_corpus$documents"), 
            robjects.r("processed_corpus$vocab"), 
            K=K, prevalence=robjects.Formula(prevalence),
            content=robjects.Formula(content),
            data=robjects.r("processed_corpus$meta"),
            init_type=init_type, seed=yield_seed(seed),
            max_em_its=max_em_its, emtol=emtol,
            verbose=verbose, reportevery=reportevery,
            LDAbeta=LDAbeta, interactions=interactions,
            ngroups=ngroups, gamma_prior=gamma_prior,
            sigma_prior=sigma_prior, kappa_prior=kappa_prior)
    elif prevalence != None and content == None:
        robjects.globalenv["fit"] = robjects.r.stm(
            robjects.r("processed_corpus$documents"), 
            robjects.r("processed_corpus$vocab"), 
            K=K, prevalence=robjects.Formula(prevalence),
            data=robjects.r("processed_corpus$meta"),
            init_type=init_type, seed=yield_seed(seed),
            max_em_its=max_em_its, emtol=emtol,
            verbose=verbose, reportevery=reportevery,
            LDAbeta=LDAbeta, interactions=interactions,
            ngroups=ngroups, gamma_prior=gamma_prior,
            sigma_prior=sigma_prior, kappa_prior=kappa_prior)
    elif prevalence == None and content != None:
        robjects.globalenv["fit"] = robjects.r.stm(
            robjects.r("processed_corpus$documents"), 
            robjects.r("processed_corpus$vocab"), 
            K=K, content=robjects.Formula(content),
            data=robjects.r("processed_corpus$meta"),
            init_type=init_type, seed=yield_seed(seed),
            max_em_its=max_em_its, emtol=emtol,
            verbose=verbose, reportevery=reportevery,
            LDAbeta=LDAbeta, interactions=interactions,
            ngroups=ngroups, gamma_prior=gamma_prior,
            sigma_prior=sigma_prior, kappa_prior=kappa_prior)
    else:
        robjects.globalenv["fit"] = robjects.r.stm(
            robjects.r("processed_corpus$documents"), 
            robjects.r("processed_corpus$vocab"), 
            K=K, data=robjects.r("processed_corpus$meta"),
            init_type=init_type, seed=yield_seed(seed),
            max_em_its=max_em_its, emtol=emtol,
            verbose=verbose, reportevery=reportevery,
            LDAbeta=LDAbeta, interactions=interactions,
            ngroups=ngroups, gamma_prior=gamma_prior,
            sigma_prior=sigma_prior, kappa_prior=kappa_prior)

    # Finish initialization
    robjects.r('''

        # Extract objects
        documents <- fit$documents
        vocab <- fit$vocab
        settings <- fit$settings 
        model <- fit$model
        verbose <- settings$verbose

        # Initialize parameters
        ngroups <- settings$ngroups
        if(is.null(model)) {

            if(verbose) cat("Beginning Initialization.\n")
            model <- stm.init(documents, settings)

            # If using the Lee and Mimno method of setting K, update the settings
            if(settings$dim$K==0) settings$dim$K <- nrow(model$beta[[1]])

            # Unpack
            mu <- list(mu=model$mu)
            sigma <- model$sigma
            beta <- list(beta=model$beta)
            if(!is.null(model$kappa)) beta$kappa <- model$kappa
            lambda <- model$lambda
            convergence <- NULL 

            # Discard the old object
            # rm(model)
        } else {

            if(verbose) cat("Restarting Model...\n")

            # Extract from a standard STM object so we can simply continue
            mu <- model$mu
            beta <- list(beta=lapply(model$beta$logbeta, exp))
            if(!is.null(model$beta$kappa)) beta$kappa <- model$beta$kappa
            sigma <- model$sigma
            lambda <- model$eta
            convergence <- model$convergence

            # Manually declare the model not converged or it will stop after the first iteration
            convergence$stopits <- FALSE
            convergence$converged <- FALSE

            # Iterate by 1 as that would have happened otherwise
            convergence$its <- convergence$its + 1 
        }    

        #Pull out some book keeping elements
        ntokens <- sum(settings$dim$wcounts$x)
        betaindex <- settings$covariates$betaindex
        stopits <- FALSE
        if(ngroups!=1) {
            groups <- cut(1:length(documents), breaks=ngroups, labels=FALSE) 
        }
        suffstats <- vector(mode="list", length=ngroups)
    ''');

    # Get the settings
    fit = dict(zip(robjects.globalenv['fit'].names, 
             list(robjects.globalenv['fit'])))
    settings = dict(zip( fit['settings'].names, 
             list(fit['settings'])))
    self.K, self.A, self.V, self.N = [int(settings['dim'][i][0]) for i in range(4)]

    # Some setup for EM, retrieving the R objects
    stopits = False
    documents = [np.array(x) for x in list(robjects.globalenv['documents'])]
    beta_index = np.array(robjects.globalenv['betaindex'])
    beta = [np.array(x) for x in robjects.globalenv['beta'][0]]
    Lambda = np.array(robjects.globalenv['lambda'])
    mu = self.__rmu_2py__(robjects.globalenv['mu'])
    sigma = np.array(robjects.globalenv['sigma'])
    verbose = settings['verbose'][0]

    # Clear the convergence object
    try:
        del convergence
        i = 0
    except:
        i = 0

    # Initiate timer
    start_time = time.clock()

    # Run EM!
    while stopits is not True:

        # Non-blocked updates
        if ngroups==1:

            # Run the E-step
            sigma_ss, beta_ss, bound_ss, Lambda = self.__estep_spark__(documents, beta_index, beta, 
                                                              Lambda, mu, sigma, verbose, False)
            if verbose:
                print("Completed E-Step, Iteration " +  str(i+1))

            # Run the M-step
            mu = self.__rmu_2py__(robjects.globalenv['opt.mu'](Lambda, 
                                 robjects.r("settings$gamma$mode"),
                                 robjects.r("settings$covariates$X"),
                                 robjects.r("settings$gamma$enet")))
            sigma = np.array(robjects.r("opt.sigma")(sigma_ss, 
                                                     Lambda, 
                                                     mu['mu'], 
                                                     float(robjects.r("settings$sigma$prior"))))
            beta_new_temp = robjects.globalenv['opt.beta'](self.__pybeta_ss_2r__(beta_ss), 
                                                           robjects.r('beta$kappa'), 
                                                           robjects.r('settings'))
            beta = [np.array(x) for x in beta_new_temp[0]]

            if verbose:
                print("Completed M-Step, Iteration " +  str(i+1))

        # Check for convergence
        try:
            convergence = robjects.globalenv['convergence.check'](bound_ss, convergence, 
                                                                  robjects.globalenv['settings'])
        except:
            convergence = robjects.globalenv['convergence.check'](bound_ss, 
                                                                  robjects.globalenv['convergence'], 
                                                                  robjects.globalenv['settings'])

        # Print updates if we have not converged
        stopits = bool(self.__rlist_2py__(convergence)['stopits'][0])
        if stopits is not True and verbose:
            robjects.r.report(convergence, robjects.r.ntokens, self.__pybeta_2r__(beta),
                     robjects.r.vocab, robjects.r('settings$topicreportevery'),
                     verbose);

        # Increase counter
        i += 1
        if verbose:
            print("")

    # Some book-keeping
    self.em_time = time.clock() - start_time
    self.trained = True

    # Save the results
    self.beta = beta
    self.logbeta = [np.log(x) for x in self.beta]
    self.Lambda = Lambda
    self.mu = mu
    self.sigma = sigma
    self.settings = settings
    self.vocab = robjects.r.vocab
    self.convergence = convergence
    self.theta = np.array(robjects.r("theta_from_lambda")(Lambda))
    self.eta = np.array(robjects.r("eta_from_lambda")(Lambda))
    self.invsigma = np.linalg.inv(sigma)